In [67]:
from pymongo import MongoClient
import pandas as pd

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client.list_database_names()

['Quotes',
 'Species',
 'admin',
 'companies',
 'config',
 'final',
 'geo',
 'labflask',
 'local']

In [4]:
companies = client.get_database("companies")

In [5]:
companies.list_collection_names()

['companies', 'offices']

In [6]:
comp = companies.get_collection("companies")

We need companies active in the `gaming industry`.

Therefore we use the function `.distinct()` to see the different types of company using the `catergory_code` parameter.

In [35]:
comp.distinct("category_code")

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

In [27]:
cond1 = {"category_code":"games_video"}

We also need companies with more than 87 employees.
* 20 Designers
* 5 UI/UX Engineers
* 10 Frontend Developers
* 15 Data Engineers
* 5 Backend Developers
* 20 Account Managers
* 1 Maintenance guy that loves basketball
* 10 Executives
* 1 CEO/President

In [34]:
cond2 = {"number_of_employees": {"$gt":87}}

In [15]:
proj = {"name":1,"offices.city":1}

In [82]:
gaming = list(comp.find({"$and": [cond1,cond2]}, proj))

In [87]:
gaming[0].get("offices")[0].get("city")

'New York'

In [92]:
empresas = []

try:
    
    for emp in gaming:

        if len(emp.get("offices")) > 1:

            for i in range(len(emp.get("offices"))):

                dicc = {
                    "Name" : emp.get("name"),
                    "city" : emp.get("offices")[i].get("city")
                }

                empresas.append(dicc)

        else:

            dicc = {
                "Name" : emp.get("name"),
                "city" : emp.get("offices")[0].get("city")
            }

            empresas.append(dicc)
            
except:
    
    None

df_gaming = pd.DataFrame(empresas)

These are the `Gaming` companies with more than 87 employees.

In [93]:
df_gaming

,Name,city
0,Livestream,New York
1,Ustream,San Francisco
2,Ustream,Los Angeles
3,Ustream,Budapest
4,RockYou,Redwood City
5,hi5,San Francisco
6,Nintendo,None
7,The Walt Disney Company,Burbank
8,Zango,Bellevue
9,hulu,Los Angeles


Developers like to be near successful tech startups that have raised at least 1 Million dollars. So we need to find those companies and where are they located.

In [48]:
cond3 = {"tag_list":{"$regex":"/*start-up*"}}

In [52]:
cond4 = {"total_money_raised":{"$gt":"$1M"}}

In [53]:
proj = {"name":1,"offices.city":1}

In [57]:
start_up = list(comp.find({"$and": [cond3,cond4]}, proj))

In [78]:
s_up=[]

for emp in start_up:
    
    if len(emp.get("offices")) > 1:
        
        for i in range(len(emp.get("offices"))):
            
            dicc = {
                "Name" : emp.get("name"),
                "city" : emp.get("offices")[i].get("city")
            }
            
            s_up.append(dicc)
        
    else:
        
        dicc = {
            "Name" : emp.get("name"),
            "city" : emp.get("offices")[0].get("city")
        }
        
        s_up.append(dicc)

df_sup = pd.DataFrame(s_up)

This are the `Start-Up` companies that have raise more than `1 Million`.

In [79]:
df_sup

,Name,city
0,Moviestorm,Cambridge
1,MojoPages,San Diego
2,MojoPages,Calabassas
3,Getyoo,Brussels
